<a href="https://colab.research.google.com/github/JudeTulel/KSA/blob/main/Marirtime_survaillance_on_YOLOV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combatting illegal maritime fishing on the Kenyan Coast
Following the call for applications by the Kenya Space agency to develop Space-Bourne solutions to address the challenges facing the Kenya, we heeded the call and developed a solution to combat illegal maritime fishing on the Kenyan Coast. The solution is a YOLO v5 model trained on a comprehensive dataset of maritime sattelite images comprising of a large-scale fine-grainted dataset of 3,435 images from various sensors, satellite platforms, locations, and seasons.

# The Dataset
For any AI or ML model data plays a paramount role. When speaking of data we talk about its quantity and quality. We chose to make use of the ShipRSImageNet datasetwhich was first presented by Z. Zhang, L. Zhang, Y. Wang, P. Feng and R. He in the paper "ShipRSImageNet: A Large-Scale Fine-Grained Dataset for Ship Detection in High-Resolution Optical Remote Sensing Images," in IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, vol. 14, pp. 8458-8472, 2021, doi: 10.1109/JSTARS.2021.3104230.

Each image is around 930×930 pixels and contains ships with different scales, orientations, and aspect ratios. The images are annotated by experts in satellite image interpretation, categorized into 50 object categories images. The fully annotated ShipRSImageNet contains 17,573 ship instances.

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [1]:
!rm -rf sample_data

In [2]:
!git clone https://github.com/ultralytics/yolov5.git  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.12 MiB | 25.39 MiB/s, done.
Resolving deltas: 100% (11856/11856), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00

In [3]:


import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.0/112.6 GB disk)


## Downloading the dataset

In [4]:
# download the zip file from google drive

!gdown --id '1wApkaSoa9mXRfXQiq6lTtlVrv4cSc6vv' # replace 'your_google_drive_file_id' with actual file id



/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1wApkaSoa9mXRfXQiq6lTtlVrv4cSc6vv
From (redirected): https://drive.google.com/uc?id=1wApkaSoa9mXRfXQiq6lTtlVrv4cSc6vv&confirm=t&uuid=60910214-dc35-4f4b-9b1d-feccd70705da
To: /content/yolov5/ShipRSImageNet_V1.zip
100% 4.58G/4.58G [01:09<00:00, 65.9MB/s]


In [5]:
# unzipping the dataset
!unzip -q ShipRSImageNet_V1.zip

In [7]:
!pwd
%cd ShipRSImageNet_V1/
!pwd

/content/yolov5
/content/yolov5/ShipRSImageNet_V1
/content/yolov5/ShipRSImageNet_V1


In [8]:

import os
import shutil
from pathlib import Path

# Define source and destination directories
source_dir = '/content/yolov5/ShipRSImageNet_V1/VOC_Format/JPEGImages/'
destination_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images/'
Path(destination_dir).mkdir(parents=True, exist_ok=True)
# Iterate over files in source directory
for filename in os.listdir(source_dir):
  # Construct full file paths
  source_path = os.path.join(source_dir, filename)
  destination_path = os.path.join(destination_dir, filename)
  # Move file
  shutil.move(source_path, destination_path)


The coco json dataset have class id starting from index 1 not zero that need to be fixed

In [9]:
import json

# Define the class names dictionary
class_names = {
    0: "AOE", 1: "Arleigh Burke DD", 2: "Asagiri DD", 3: "Atago DD", 4: "Austin LL",
    5: "Barge", 6: "Cargo", 7: "Commander", 8: "Container Ship", 9: "DOCK",
    10: "Enterprise", 11: "EPF", 12: "Ferry", 13: "Fishing Vessel", 14: "Hatsuyuki DD",
    15: "Hovercraft", 16: "Hyuga DD", 17: "LHA LL", 18: "LSD 41 LL", 19: "Masyuu AS",
    20: "Medical ship", 21: "Midway", 22: "Motorboat", 23: "Nimitz", 24: "Oil Tanker",
    25: "Osumi LL", 26: "Other Aircraft Carrier", 27: "Other Auxiliary Ship",
    28: "Other Destroyer", 29: "Other Frigate", 30: "Other Landing", 31: "Other Merchant",
    32: "Other Ship", 33: "Other Warship", 34: "Patrol", 35: "Perry FF",
    36: "RORO", 37: "Sailboat", 38: "Sanantonio AS", 39: "Submarine", 40: "Test ship",
    41: "Ticonderoga", 42: "Training ship", 43: "Tugboat", 44: "Wasp LL", 45: "Yacht",
    46: "YuDao LL", 47: "YuDeng LL", 48: "YuTing LL", 49: "YuZhao LL"
}

def adjust_coco_categories(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Adjust category ids in 'categories'
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id - 1
        category['id'] = new_id
        category['name'] = class_names[new_id]

    # Adjust category ids in 'annotations'
    for ann in data['annotations']:
        ann['category_id'] -= 1

    # Save the adjusted JSON file
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

# Adjust category IDs in the dataset JSON files
json_files = [
    '/content/yolov5/ShipRSImageNet_V1/COCO_Format/ShipRSImageNet_bbox_train_level_3.json',
    '/content/yolov5/ShipRSImageNet_V1/COCO_Format/ShipRSImageNet_bbox_val_level_3.json',
]

for json_file in json_files:
    adjust_coco_categories(json_file)

## Converting COCO JSON to YOLOtxt format


In [10]:
import json
import os
from pathlib import Path

def coco_to_yolo(coco_json, images_dir, labels_dir):
    # Load COCO JSON
    with open(coco_json, 'r') as f:
        coco_data = json.load(f)

    # Create directories if they don't exist
    Path(labels_dir).mkdir(parents=True, exist_ok=True)

    # Mapping of category ID to category name
    category_map = {cat['id']: cat['name'] for cat in coco_data['categories']}

    # Process each image
    for image in coco_data['images']:
        image_id = image['id']
        image_filename = image['file_name']
        image_path = os.path.join(images_dir, image_filename)
        txt_filename = os.path.splitext(image_filename)[0] + '.txt'
        txt_path = os.path.join(labels_dir, txt_filename)

        # Open the corresponding .txt file for writing
        with open(txt_path, 'w') as txt_file:
            # Get annotations for this image
            annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

            for ann in annotations:
                # Convert COCO bbox format [x, y, width, height] to YOLO format [x_center, y_center, width, height]
                bbox = ann['bbox']
                x, y, width, height = bbox
                x_center = (x + width / 2) / image['width']
                y_center = (y + height / 2) / image['height']
                width /= image['width']
                height /= image['height']

                # Get the class ID and map it to a class name
                class_id = ann['category_id']
                class_name = category_map[class_id]

                # Write to the .txt file in YOLO format
                txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Parameters
coco_json = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/ShipRSImageNet_bbox_train_level_3.json'  # Path to your COCO JSON file
images_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images'  # Path to your images directory
labels_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/labels/train'  # Path to the directory where YOLO .txt files should be saved

coco_to_yolo(coco_json, images_dir, labels_dir)


In [11]:
#for validation data

val_json =   '/content/yolov5/ShipRSImageNet_V1/COCO_Format/ShipRSImageNet_bbox_val_level_3.json'
images_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images'
labels_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/labels/val'
coco_to_yolo(val_json, images_dir, labels_dir)


In [12]:

import os
import shutil

# Define source and destination directories
source_dir =     '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images'  # Directory containing all images
val_labels_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/labels/val'  # Directory with validation labels
destination_dir ='/content/yolov5/ShipRSImageNet_V1/COCO_Format/images/val'  # Directory to move validation images

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get a list of validation image filenames from the labels
val_image_filenames = []
for filename in os.listdir(val_labels_dir):
    if filename.endswith('.txt'):
        val_image_filenames.append(filename.replace('.txt', '.bmp'))

# Move validation images
for filename in val_image_filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    if os.path.exists(source_path):  # Check if the image exists
        shutil.move(source_path, destination_path)


In [13]:
#moving the image for train to train folder
import os
import shutil

# Define source and destination directories
source_dir =      '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images'  # Directory containing all images
val_labels_dir =  '/content/yolov5/ShipRSImageNet_V1/COCO_Format/labels/train'  # Directory with validation labels
destination_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images/train'  # Directory to move validation images
# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get a list of validation image filenames from the labels
val_image_filenames = []
for filename in os.listdir(val_labels_dir):
    if filename.endswith('.txt'):
        val_image_filenames.append(filename.replace('.txt', '.bmp'))

# Move validation images
for filename in val_image_filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    if os.path.exists(source_path):  # Check if the image exists
        shutil.move(source_path, destination_path)


In [14]:
# prompt: read the file names in test.txt and move image files with matching names to /content/yolov5/yolov5/ShipRSImageNet_V1/COCO_Format/labels/test

# Read filenames from test.txt
with open('/content/yolov5/ShipRSImageNet_V1/COCO_Format/test.txt', 'r') as f:
    test_filenames = [line.strip() for line in f]

# Define source and destination directories
source_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images'
destination_dir = '/content/yolov5/ShipRSImageNet_V1/COCO_Format/images/test'

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move test images
for filename in test_filenames:
    source_path = os.path.join(source_dir, filename + '.bmp')  # Assuming images are .bmp
    destination_path = os.path.join(destination_dir, filename + '.bmp')
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)


# Training the model
Now the the dataset is completly downloaded an extracted into the yolov5 folder we will procced with training

We chose to train it

In [16]:
%cd ..

/content/yolov5


In [22]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 960 --batch 4 --epochs 16 --data VOC1.yaml --weights yolov5s.pt --cache

2025-03-17 22:40:35.044020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742251235.265573   28394 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742251235.327039   28394 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=VOC1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=16, batch_size=4, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok

# Detect

`detect.py` runs YOLOv5 inference on a variety of sources and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [21]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 960 --conf 0.20 --source /content/yolov5/ShipRSImageNet_V1/COCO_Format/images/000019.bmp


detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/yolov5/ShipRSImageNet_V1/COCO_Format/images/000019.bmp, data=data/coco128.yaml, imgsz=[960, 960], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7144975 parameters, 0 gradients, 16.2 GFLOPs
image 1/1 /content/yolov5/ShipRSImageNet_V1/COCO_Format/images/000019.bmp: 960x960 3 AOEs, 2 Arleigh Burke DDs, 22.1ms
Speed: 0.9ms pre-process, 22.1ms inference, 127.1ms NMS per image at shape (1, 3, 960, 960)
Results saved to runs/detect/exp4


## Local Logging

Training results are automatically logged with [Tensorboard](https://www.tensorflow.org/tensorboard) and [CSV](https://github.com/ultralytics/yolov5/pull/4148) loggers to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc.

This directory contains train and val statistics, mosaics, labels, predictions and augmentated mosaics, as well as metrics and charts including precision-recall (PR) curves and confusion matrices.

<img alt="Local logging results" src="https://user-images.githubusercontent.com/26833433/183222430-e1abd1b7-782c-4cde-b04d-ad52926bf818.jpg" width="1280"/>


# Exporting ONNX



In [ ]:
!python export.py --weights /content/best.pt --include onnx --simplify

export: data=data/coco128.yaml, weights=['/content/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=True, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7144975 parameters, 0 gradients, 16.2 GFLOPs

PyTorch: starting from /content/best.pt with output shape (1, 25200, 55) (14.3 MB)

ONNX: starting export with onnx 1.17.0...
ONNX: slimming with onnxslim 0.1.35...
ONNX: export success ✅ 1.5s, saved as /content/best.onnx (27.7 MB)

Export complete (2.5s)
Results saved to /content
Detect:          python detect.py --weights /content/best.onnx 
Validate:        python val.py --weights /content/best.onnx 
PyTorch Hub:     model = torch.hub.load('ultra

In [ ]:
!zip -r /content/best.onnx.zip /content/best.onnx


  adding: content/best.onnx (deflated 16%)


In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
import onnxruntime as ort
import numpy as np
import cv2

def preprocess_image(image_path, input_size=(640, 640)):
    """Load and preprocess an image for ONNX inference."""
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, input_size)
    image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
    image_normalized = image_rgb.astype(np.float32) / 255.0
    image_transposed = np.transpose(image_normalized, (2, 0, 1))
    image_batched = np.expand_dims(image_transposed, axis=0)
    return image_batched, image  # Preprocessed and original image

def postprocess_output(output, input_size=(640, 640), conf_threshold=0.5, iou_threshold=0.45):
    """Decode YOLO model output and filter detections."""
    predictions = output[0]  # (1, 25200, 55)
    predictions = predictions[0]  # Remove the batch dimension -> (25200, 55)

    # Extract bounding box coordinates, object confidence, and class scores
    box_coords = predictions[:, :4]  # x_center, y_center, width, height
    object_conf = predictions[:, 4:5]  # Objectness confidence
    class_scores = predictions[:, 5:]  # Class probabilities

    # Compute final scores: object confidence * class scores
    scores = object_conf * class_scores  # Shape: (25200, 50)

    # Filter out low-confidence detections
    detections = []
    for i in range(predictions.shape[0]):
        class_id = np.argmax(scores[i])  # Get class with the highest score
        confidence = scores[i, class_id]
        if confidence > conf_threshold:
            x_center, y_center, width, height = box_coords[i]
            x1 = int((x_center - width / 2) * input_size[0])
            y1 = int((y_center - height / 2) * input_size[1])
            x2 = int((x_center + width / 2) * input_size[0])
            y2 = int((y_center + height / 2) * input_size[1])
            detections.append([x1, y1, x2, y2, confidence, class_id])

    # Apply Non-Maximum Suppression
    return nms(detections, iou_threshold)

def nms(detections, iou_threshold=0.45):
    """Apply Non-Maximum Suppression to reduce overlapping detections."""
    detections = sorted(detections, key=lambda x: x[4], reverse=True)  # Sort by confidence
    final_detections = []

    while detections:
        best_box = detections.pop(0)
        final_detections.append(best_box)
        detections = [
            box for box in detections
            if iou(best_box, box) < iou_threshold
        ]
    return final_detections

def iou(box1, box2):
    """Calculate Intersection over Union (IoU) between two boxes."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    return inter_area / (box1_area + box2_area - inter_area)


def nms(detections, iou_threshold=0.45):
    """Apply Non-Maximum Suppression."""
    detections = sorted(detections, key=lambda x: x[4], reverse=True)  # Sort by confidence
    final_detections = []

    while detections:
        best_box = detections.pop(0)
        final_detections.append(best_box)
        detections = [
            box for box in detections
            if iou(best_box, box) < iou_threshold
        ]
    return final_detections

def iou(box1, box2):
    """Calculate Intersection over Union (IoU) between two boxes."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    return inter_area / (box1_area + box2_area - inter_area)

def draw_bounding_boxes(image, detections, input_size):
    """Draw bounding boxes on the image."""
    for x1, y1, x2, y2, confidence, class_id in detections:
        label = f"Class {int(class_id)}: {confidence:.2f}"
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

def run_inference(model_path, image_path):
    """Run inference on a single image."""
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    input_shape = session.get_inputs()[0].shape[2:]  # HxW

    input_data, original_image = preprocess_image(image_path, input_size=tuple(input_shape))

    outputs = session.run(None, {input_name: input_data})

    results = postprocess_output(outputs, input_size=input_shape, conf_threshold=0.2)
    return draw_bounding_boxes(original_image, results, input_shape)

if __name__ == "__main__":
    model_file = "/content/ShipDetectionClassifier.onnx"
    test_image = "/content/yolov5/ShipRSImageNet_V1/VOC_Format/JPEGImages/000019.bmp"
    output_image_path = "/content/output_with_boxes.bmp"

    image_with_boxes = run_inference(model_file, test_image)
    cv2.imwrite(output_image_path, image_with_boxes)
    print(f"Saved output image with bounding boxes to {output_image_path}")


Saved output image with bounding boxes to /content/output_with_boxes.bmp
